# FY4A 农业 L1 读取器介绍

## 1. 支持文件

FY4A AGRI数据的NetCDF格式。
完整磁盘和区域镜像均受支持。
示例文件名：
完整磁盘:    
FY4A-_-AGRI-_-N_DISK_1047E_L1-_-FDI-_-多光谱标准_20190807060000_20190807061459_4000米_版本1.HDF
REGC：
FY4A-_AGRI--_N_REGC_1047E_L1-_FDI-_MULT_NOM_20190807045334_20190807045750_1000M_V0001.HDF
*全盘扫描由 DISK 标识，区域扫描由 REGC 标识。*
数据链接:
实时数据服务（30天）和简介文件：        https://fy4.nsmc.org.cn/data/en/data/realtime.html
历史数据 (2018-03-12 -- )：        http://satellite.nsmc.org.cn/PortalSite/Data/Satellite.aspx
FY4A 官方气象应用平台:        http://rsapp.nsmc.org.cn/geofy/

## 2. 校准

您有三个选项：
1. 原始探测器计数 (所有通道)
2. 反射率 (C01 - C06)
3. 辐射亮度和亮度温度 (C07 - C14)

## 3. 示例

### 安装

```
$ conda install -c conda-forge satpy
```

### 加载数据

In [1]:
import os, glob
from satpy.scene import Scene

In [2]:
# load FY4A filenames
filenames = glob.glob('/xin/data/FY4A/20190807/FY4A-_AGRI*4000M_V0001.HDF')

# create the scene object
scn = Scene(filenames, reader='agri_l1')

# check available channels
scn.available_dataset_names()

['C01',
 'C02',
 'C03',
 'C04',
 'C05',
 'C06',
 'C07',
 'C08',
 'C09',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'satellite_azimuth_angle',
 'satellite_zenith_angle',
 'solar_azimuth_angle',
 'solar_glint_angle',
 'solar_zenith_angle']

In [3]:
# take the ir channel as example
ir_channel = 'C12'
scn.load([ir_channel], generate=False, calibration='brightness_temperature')

### 生成完整磁盘镜像

In [4]:
# display in notebook
scn.show(ir_channel)

<img src='./figures/agri_C12.png'>

In [5]:
# save to file
# scn.save_dataset(ir_channel, filename='{sensor}_{name}.png')

### 生成完整磁盘镜像 (真彩色)

In [6]:
# get a list of all available composites for the current scene
scn.available_composite_names()

Too many possible datasets to load for 3.9
Too many possible datasets to load for 3.9
Too many possible datasets to load for DatasetID(name=None, wavelength=3.9, resolution=None, polarization=None, calibration=None, level=None, modifiers=())
Too many possible datasets to load for 3.9
Too many possible datasets to load for 3.9
Too many possible datasets to load for DatasetID(name=None, wavelength=3.9, resolution=None, polarization=None, calibration=None, level=None, modifiers=())
Too many possible datasets to load for 3.9


['ash',
 'dust',
 'fog',
 'green',
 'green_snow',
 'ir108_3d',
 'ir_cloud_day',
 'natural_color',
 'natural_color_sun',
 'overview',
 'overview_sun',
 'true_color']

In [7]:
# Beware that this step might need much memory available on the processing machine (depending on the number of cpu cores)
# You can check the link of FAQ about memory:
#    https://satpy.readthedocs.io/en/latest/faq.html

composite = 'true_color'
scn.load([composite])
scn.show(composite)
# scn.save_dataset(composite, filename='{sensor}_{name}.png')

Required file type 'agri_l1_4000m_geo' not found or loaded for 'solar_zenith_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'satellite_zenith_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'satellite_azimuth_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'solar_azimuth_angle'
Inconsistent sensor/satellite input - sensor set to agri
Inconsistent sensor/satellite input - sensor set to agri


<img src='./figures/agri_true_color.png'>

### 在感兴趣区域内生成合成图像

- 我以台风LEKIMA为例。
我们可以定义一个地图投影和一个子区域，并将数据投影到该区域上。
``Pyresample`` 可以轻松地定义区域。
此定义也可以配置在 `area.yaml` 配置文件中。

In [8]:
from pyresample import get_area_def

area_id = 'lekima'

x_size = 549
y_size = 499
area_extent = (-1098006.560556, -967317.140452, 1098006.560556, 1026777.426728)
projection = '+proj=laea +lat_0=19.0 +lon_0=128.0 +ellps=WGS84'
description = "Typhoon Lekima"
proj_id = 'laea_128.0_19.0'

areadef = get_area_def(area_id, description, proj_id, projection,x_size, y_size, area_extent)

你可以通过 [coord2area_def.py](https://github.com/pytroll/satpy/blob/main/utils/coord2area_def.py) 轻松生成区域。
以下是 `python coord2area_def.py lekima_4km laea 10 28 118 138 4` 的输出：
```
lekima_4km:
  description: lekima_4km
  projection:
    proj: laea
    ellps: WGS84
    lat_0: 19.0
    lon_0: 128.0
  shape:
    height: 499
    width: 549
  area_extent:
    lower_left_xy: [-1098006.560556, -967317.140452]
    upper_right_xy: [1098006.560556, 1026777.426728]
```

现在，您可以将配置添加到 `$PPP_CONFIG_DIR/areas.yaml` 并直接使用。

In [9]:
# If you have added it to areas.yaml, you can use the name directly:
os.environ['PPP_CONFIG_DIR'] = '/yin_raid/xin/satpy_config/'
lekima_scene = scn.resample('lekima_4km')

# Otherwise, you need to use the areadef defined above:
# lekima_scene = scn.resample(areadef)

In [10]:
lekima_scene.show(composite)
# lekima_scene.save_dataset(composite, filename='{sensor}_{name}_resampled.png')

<img src='./figures/agri_true_color_resampled.png'>

如果你想生成具有特定颜色映射的图像，如下面的图所示，
请查看有关 `enhancement` 的另一个笔记本。

<img src='./figures/agri_C12_resampled_colorize.png'>